# assign random direction to image and download #
**Author:** Andrew Larkin

**Date Created:** December 16, 2019

**Summary:** Given heading of the road an image was taken from, randomly assign a cardinal viewing direction (front, left, back, right), and determine the compass heading needed to download image.  In the second part of the script, given image id and heading, download image.  

### setup ###

In [3]:
import os
import pandas as ps
import random
import wget
import time
import globalConstants as gConst

In [ ]:
PARENT_FOLDER = gConst.PARENT_FOLDER
OUTPUT_FOLDER = PARENT_FOLDER + "Downloaded_Images/"
INPUT_CSV = PARENT_FOLDER + "olympiaPoints.csv"
UPDATED_CSV = PARENT_FOLDER + "olympiaPoints_updated.csv"
ANGLE_MULTIPLE = 90
API_KEY = gConst.API_KEY

INDEX = 5
SAMPLE_SIZE = 20000

### load raw data used to calculate headings ###
**Inputs:** <br>
- **inFilepath** (str) - absolute filepath to csv data

**Outputs:**
pandas dataframe, sorted by FID

In [4]:
def loadSamplePoints(inFilepath):
    rawData = ps.read_csv(inFilepath)
    return(rawData.sort_values(by=['FID']))

### randomly assign a viewing angle to an image and determine the compass direction that corresponds to the angle ###
**Inputs:**
- **origHeading** (int) - compass heading to corresponds to a viewing angle of straight ahead

**Outputs:**
- **cat** (str) - categorical viewing angle (straight ahead or side)
- **downloadHeading** (int) - compass direction of the viewing angle

In [5]:
def randomizeHeading(origHeading):
    randInt = random.randint(0,3)
    cat = "none"
    if(randInt %2 == 0):
        cat = "straightAngle"
    else:         
        cat = "sideAngle"
    downloadHeading = (origHeading + randInt*ANGLE_MULTIPLE)%360
    return([cat,downloadHeading])

### randomly assign a viewing angle and determine compass directions for all images in a set ###
**Inputs:** 
- **headings** (list of strings) - list of compass headings that corresponds to a viewing angle of straight ahead for all images in the set

**Outputs:**
- **cats** (list of strings) - list of categorical viewing angles (straight ahead or side)
- **downloadHeadings** (list of ints) - list of compass directions of the viewing angles

In [6]:
def randomizeAllHeadings(headings):
    downloadHeadings, cats = [],[]
    for heading in headings:
        tempCat,tempHeading = randomizeHeading(heading)
        downloadHeadings.append(tempHeading)
        cats.append(tempCat)
    return([cats,downloadHeadings])

### main function for randomly assigning headings ###

In [7]:
def main():
    samplingData = loadSamplePoints(INPUT_CSV)
    cats,headings =  randomizeAllHeadings(samplingData['viewingAngle'])
    samplingData['viewCat'] = cats
    samplingData['downloadHeading'] = headings
    samplingData.to_csv(UPDATED_CSV,index=False)

In [12]:
main()

### create wget string for downloading streetview image ###
**Inputs:**
- **panid** (str) - unique GSV id
- **heading** (int) - viewing angle to center the image on 

**Outputs**
- **combinedString** - wget string for downloading image

In [8]:
def createDownloadString(panid,heading):
    baseString = "https://maps.googleapis.com/maps/api/streetview?size=640x640&pitch=0"
    locString = "&pano=" + str(panid)
    headingString = "&heading=" + str(heading)
    apiString = "&key=" + str(API_KEY)
    combinedString = baseString + locString + headingString + apiString
    return(combinedString)

### determine absolute filepath to store a downloaded image ###
**Inputs:**
- **panid** (str) - unique GSV image id
- **heading** (int) - viewing angle downloaded image is centered on

**Outputs:**
- absolute filepath to store downloaded image, in string format

In [9]:
def createDownloadFilepath(panid,heading):
    return(OUTPUT_FOLDER + panid + "_" + str(heading) + ".jpg")

In [16]:
downloadData = ps.read_csv(UPDATED_CSV)
downloadData = downloadData.sort_values(by=['FID'])
startIndex = INDEX*SAMPLE_SIZE
endIndex = min((INDEX+1)*SAMPLE_SIZE,len(downloadData['FID']))
sampleToDownload = downloadData#downloadData.iloc[startIndex:endIndex]
print(sampleToDownload)
for index in range(SAMPLE_SIZE):
    oneSample = sampleToDownload.iloc[index]
    downloadUrl = createDownloadString(oneSample['id'],oneSample['downloadHeading'])
    outputFilepath = createDownloadFilepath(oneSample['id'],oneSample['downloadHeading'])
    if not(os.path.exists(outputFilepath)):
        try:
            wget.download(downloadUrl,outputFilepath)
        except:
            print("couldn't download image: " + outputFilepath)

       FID  Join_Count   distance                      id        lat  \
0        0           1  26.386666  2kGxPAk72vAX7Bz9OgJrQg  47.056922   
1        1           1  17.671732  eZ3mpFHW5rC7Zzjd1lSzTQ  47.066785   
2        2           1  30.564218  k_4e5iS5MOK5owMMFBSwMw  46.959897   
3        3           1  11.600742  oEPuWaB4sL-DVLL6lTqMDA  47.025545   
4        4           1  28.653638  HzY-Dg_Tc8DKFxyi8FO-UA  47.035386   
...    ...         ...        ...                     ...        ...   
3379  3891           1  22.120900  hNFOTwOHiXwgUfPKgXO3gA  47.013768   
3380  3892           1  19.954747  fy25Y2GFJG8PknW1QeQvwg  47.058809   
3381  3893           1  35.016106  Ajn0q5cgNx-PXW5fhUEg8Q  47.024248   
3382  3896           1  17.075390  2XHuTlU3Os-qwKj-y2RIlA  47.032267   
3383  3897           1  19.378979  0mSP8vmw3bg-KvBTRmLDtw  47.090456   

             lon  month  year  viewingAngle        viewCat  downloadHeading  
0    -122.825596      9  2011           106      sideAngl

IndexError: single positional indexer is out-of-bounds